In [1]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from google.colab import files

# Check GPU availability
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

print(f"Using device: {device}")

Using device: cuda:0


In [3]:
# Define the model ID
model_id = "openai/whisper-large-v3"

# Load the model and processor
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

print("Model and processor loaded successfully!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Model and processor loaded successfully!


In [4]:
# Initialize the pipeline for transcription
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,  # Split audio into 30-second chunks
    batch_size=4,  # Adjust batch size for resource constraints
    torch_dtype=torch_dtype,
    device=device,
)

print("Pipeline initialized successfully!")

Device set to use cuda:0


Pipeline initialized successfully!


In [5]:
audio_file = '/content/GLT1917751681.mp3'

In [7]:
# Set task to translate to English
result = pipe(audio_file, generate_kwargs={"task": "translate"})

# Print the transcription
print("Transcription (Translated to English):")

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=translate, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=translate.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription (Translated to English):


In [15]:
text = result["text"]

## T5-medium

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "ashwin0211/lora_t5_medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:
def generate_summary(text, model, tokenizer, max_length=150, min_length=40, do_sample=False):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

transcript = "Your podcast transcript goes here..."
summary = generate_summary(text, model, tokenizer)
print(summary)

'Opening moves' is a simple way to start conversations with your matches. It's a simple way to choose a question and let your matches reply to kick off the once. It's a simple way to make the first move or not. It's entirely up to you. With the PayPal debit card, just tap to pay in store with your digital wallet and earn 5% cash back on purchases.


## T5-small

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "RahulMandadi/lora_T5_small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.45M [00:00<?, ?B/s]

In [19]:
def generate_summary(text, model, tokenizer, max_length=150, min_length=40, do_sample=False):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

transcript = "Your podcast transcript goes here..."
summary = generate_summary(text, model, tokenizer)
print(summary)

I'm going to be sharing with you all of the gross and weird shit that I do when I'm alone to make you feel less gross and weird about yourself and to hopefully entertain you and to get it off my chest. I'm going to be sharing with you all of the gross and weird shit that I do when I'm alone to make you feel less gross and weird about yourself and to make you feel less gross and weird about yourself and to hopefully entertain you and to get it off my chest. This episode is brought to you by Bumble.


## Bart-large

In [21]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "ashwin0211/bart-large-cnn-repo"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [22]:
def generate_summary(text, model, tokenizer, max_length=150, min_length=40, do_sample=False):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

transcript = "Your podcast transcript goes here..."
summary = generate_summary(text, model, tokenizer)
print(summary)

. This episode is brought to you by Airbnb and Bumble. This week's episode is about the grossest and weirdest things that I do when I'm alone. I'm going to be sharing with you all of the gross and weird things I do in my moments alone to make you feel less gross or weird about yourself and to hopefully entertain you and to get it off my chest. I like to be in a relationship, but I like casually dating. Bumble is helping take some of the pressure off by introducing a new feature that lets you choose to make the first move or not.


## Bart-small

In [23]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "RahulMandadi/lora_Bart_base_cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

In [24]:
def generate_summary(text, model, tokenizer, max_length=150, min_length=40, do_sample=False):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

transcript = "Your podcast transcript goes here..."
summary = generate_summary(text, model, tokenizer)
print(summary)

This episode is brought to you by Bumble. Bumble is helping take some of the pressure off the dating stage with its new feature, opening moves. It's a simple way to start conversations. Just choose a question and let your matches reply to kick off the once. Running errands, grabbing coffee, online shopping at 2 a.m. for like who knows what, getting some work done, but then getting distracted and then going back to work and now, oh my God, it's chaos, but it's our chaos. Because PayPal makes it super easy to pay for everything you need everywhere you go. And now


On analyzing the summaries bart performs better than t5 model. Only the issue with bart is it's small input token size.